## 高斯消元法or BP 主要是解码算法，或者未来使用的那些升级算法。 主要参考 主要是异或方程组
## 一定要注意这个是异或方程组。
下面对应说明
增广矩阵，A*X=Y,,,      
A对应编码矩阵，n*k，生成n个编码行向量。
X是k个原始行向量。
进行的运算是异或运算。

### 仿真结构
1. source_code
包含产生的原始数据，信息长度length_bytes(8*length)，原始信息的个数k 
使用vscode里面的hextemp可以查看

2. 

In [126]:
import random
import struct
import numpy as np
from math import *
from collections import defaultdict
class source_code:

    def __init__(self,k,length_bytes,file_name="source_code"):
        self.k=k
        self.length_bytes=length_bytes
        self.file_name=file_name
        self.__generate_random_source_code()

    def __generate_random_source_code(self):
        '''
        generate k*8Llength_bytes bits bin_file
        @parameter k :numer of source code 
        @parmamter file_name="source_code" by default
        @parmeter length_bytes 8*length_bits
        
        '''
        k=self.k
        length_bytes=self.length_bytes
        file_name=self.file_name
        code_list=[struct.pack("B",i)  for i in range(256)]

        with open(file_name,"wb") as fd:
            for i in range(k*length_bytes):
                fd.write(code_list[random.randint(0,255)])
class encoder:
    def __init__(self,k,source_code,length_bytes,file_name="encoding_data"):
        self.k=k
        self.file_name=file_name
        self.source_code=source_code
        self.length_bytes=length_bytes
        self.read_source()
    def read_source(self,):
        source_code=self.source_code
        length_bytes=self.length_bytes
        k=self.k
        #self.data=np.zeros((k,length_bytes),dtype=int)
        with open("source_code_test","rb") as fd:
            one_source_code=fd.read(length_bytes)
            self.data=np.array(list(one_source_code),dtype="u1").reshape(1,length_bytes)
            count=1
            while(1):
                one_source_code=fd.read(length_bytes)
                if not one_source_code:
                    break
                new_line=np.array(list(one_source_code),dtype="u1").reshape(1,length_bytes)
                self.data=np.concatenate((self.data,new_line))
                count+=1
        if count!=self.k:
            raise Exception("the source code not match the size")

        
        pass
    def load_A_encoding(self,A_name):
        file_name=self.file_name+"load_A"
        A=self.__encoding_load_A(A_name)
        self.__encoding(A,file_name)
        print(A.shape)
        print(self.data.shape)
    def __encoding_load_A(self,A_name):
        k=self.k
        A=None
        length=floor(k/8)
        with open(A_name,"rb") as fd:
            while(1):
                new_line=list(fd.read(length))
                if not(new_line):
                    break
                if A is None:
                    A=np.array(new_line,dtype="u1").reshape(1,length)
                else:
                    A=np.concatenate((A,np.array(new_line,dtype="u1").reshape(1,length)),axis=0)
            
            
        return A

        pass
    def random_encode(self,n):
        file_name=self.file_name+"random"
        A=self.__random_A(n)
        self.__encoding(A,file_name)
        print(A.shape)
        print(self.data.shape)
        pass
    def __random_A(self,n):
        k=self.k
        A=np.random.randint(0,256,(n,floor(k/8)),dtype="u1")
        return A
        pass
    def __encoding(self,A,file_name):
        '''
        AX=Y, A:(n,k)  X(k,length_bytes) Y(n,length_bytes)
        这里有点麻烦，因为还要写入编码矩阵。因为要按照bit写入，所以应该写入ceil(n,8),最后一个，的后
        几位是没有用到，但是写这里。
        '''
        k=self.k
        n=A.shape[0]
        length_bytes=self.length_bytes
        with open(file_name,"wb") as fd:
            for i in range(n):
                new_line=np.zeros((length_bytes),dtype="u1")
                number=0
                for j in range(floor(k/8)):
                    esi_bytes=A[i][j]
                    fd.write(struct.pack("B",esi_bytes))
                    for v in range(8):
                        if number>=k:
                            break
                        if esi_bytes&(1<<7):
                            new_line=new_line^self.data[number]
                        esi_bytes=esi_bytes<<1
                        number+=1
                new_line=[struct.pack("B",bytes_newline)for bytes_newline in new_line]
                #print(new_line)
                fd.write(b''.join(new_line))
            



        
        
       

    def rs_encode(self):

        file_name=self.file_name+"rs"
        pass
class bec_channel:
    def __init__(self,loss):
        self.loss=loss
        
    def transfer(self,k,length_bytes,encoder_data_file,file_name="after_bec_encoder",seed=10):
        
        random.seed(seed)
        length=length_bytes+floor(k/8)
        threshold=10000*self.loss
        '''
        this is the file name 
        '''
       # print(encoder_data_file)
        with open(encoder_data_file,"rb") as fd1, open(file_name,"wb") as fd2:
            while(1):
                message=fd1.read(length)
                if not message:
                    break
                success=(random.randint(0,10000)>threshold)
                if success:
                    fd2.write(message)

class decoder:
    def __init__(self,k,length_bytes,bec_encode_data_file,file_name="decoder_data",source_data_file=None):
        self.file_name=file_name
        self.k=k
        self.n=n
        self.bec_encode_data_file=bec_encode_data_file
        self.source_data_file=source_data_file
        self.length_bytes=length_bytes
        self.length_encoder=floor(k/8)+length_bytes
        self.__read_encoder_data()
        self.decoder_data=np.zeros((k,length_bytes),dtype="u1")
        
    def __read_encoder_data(self,):
        '''
        从encoder data 中提取两个变量一个是A，一个是Y
        '''
        k=self.k
        length_bytes=self.length_bytes
        head=floor(k/8)
        length=head+length_bytes
        A=None
        source_data=None
        with open(self.bec_encode_data_file,"rb") as fd1:
            while(1):
                message=fd1.read(length)
                if not message:
                    break
                message=list(message)
                if A is None:
                    A=np.array(message[:head],dtype="u1").reshape(1,head)
                    source_data=np.array(message[head:],dtype="u1").reshape(1,length_bytes)
                    continue
                else:
                    new_line_A=np.array(message[:head],dtype="u1").reshape(1,head)
                    new_line_source_data=np.array(message[head:],dtype="u1").reshape(1,length_bytes)
                    A=np.concatenate((A,new_line_A),axis=0)
                    source_data=np.concatenate((source_data,new_line_source_data),axis=0)
        
        self.A=A
        self.encoder_data=source_data
        print(A.shape[0])

    def gas_eli(self,):
        pass
    def bp(self):
        '''
        需要用到图论，所以需要构建两个集合，也就是两个dict，分别代表左右两边的节点
        
        dic_source——node，和该source_node 当该source_node恢复了，
        
        dic_，挑选当前度节点为1的节点，然后可以自己创建，也可以使用别人的
        首先要有一个度的dic,实现每个度，都能找到对应的集合，encoder_序号
        然后，因为度为1,的节点里面存的是哪个原始souce序号，需要一个dic，查找这个，就知道原始信息序号有什么。相当于是encoder序号映射source序号
        然后需要一个集合，所有相应的该source序号，连接的encoder序号。
        然后对于每个encoder，剔除该source，并更新。
        所以需要三个dic，对应关系，如上。流程如上。
        然后把所有的和这个原始的都剔除掉。
        '''
        totol_source_number=self.k
        totol_encoder_number=self.A.shape[0]
        head=floor(self.k/8)
        dic_degree2encode=defaultdict(set)###degree 是encoder的degree
        dic_encode2source=defaultdict(set)
        dic_source2encode=defaultdict(set)
        '''
        初始化3个dic，然后开始bp姨妈算法
        '''
        for encoder_number in range(totol_encoder_number):
            count=0
            for head_bytes in range(head):
                key=self.A[encoder_number][head_bytes]
                
                for pos in range(8):
                    source_number=pos+head_bytes*8
                    if source_number>=k:
                        break
                    if key&(1<<(7-pos)):
                        dic_encode2source[encoder_number].add(source_code)
                        dic_source2encode(source_number).add(source_code)
                        count+=1
                    else:
                        continue
            dic_degree2encode[count].add(encoder_number)
        source_recover_number=0
        while(1):
            if source_recover_number==totol_source_number:
                self.__write_encoder_data(self.file_name+"_bp")
            if len(dic_degree2encode[1])==0:
                print(f"{source_recover_number/totol_source_number*100}% sysmle has been recoverd")
            now_degree_one=dic_degree2encode[1].pop()
            source_number_new=dic_encoder2source[now_degree_one].pop()
            self.decoder_data[source_number_new]=self.encoder_data[now_degree_one]
            for encoder_number_connect_source_new in dic_source2encode[source_number_new]:
                if encoder_number_connect_source_new==now_degree_one:
                    continue
                else:
                    '''
                    dic_degree2encode 减少度
                    dic_encode2source 删掉边
                    '''
                    self.encoder_data[encoder_number_connect_source_new]=self.encoder_data[encoder_number_connect_source_new]^self.encoder_data[now_degree_one]
                    dic_encode2source[encoder_number_connect_source_new].remove(source_number_new)
                    ll=len(encode2source[encoder_number_connect_source_new])
                    dic_degree2encode[ll].add(encoder_number_connect_source_new)
                    dic_degree2encode[ll+1].remove(encoder_number_connect_source_new)

    def __write_encoder_data(self,file_name):
        with open(file_name,"wb") as fd:
            for i in range(k):
                new_line=[struct.pack("B",bytes_source)for bytes_source in self.decoder_data[i]]
                #print(new_line)
                fd.write(b''.join(new_line))
        

        
    def check(self):
        source_data_file=self.source_data_file
        decoder_data_file=self.file_name
        length=self.length_bytes
        with open(source_data_file,"rb") as fd1,open(decoder_data_file,"rb") as fd2:
            while(1):
                message_source=fd1.read(length)
                message_decoder=fd2.read(length)
                if not message_source and not message_decoder:
                    break
                elif message_decoder!=message_source:
                    return False
                else:
                    continue
        return True


        

IndentationError: expected an indented block (<ipython-input-126-396646447e73>, line 241)

In [87]:
## file name of the source code
source_code_file="source_code_test"
#number of source _code
k=128
##length of the code
length_bytes=3
##initial the generate the source code 
sc=source_code(k,length_bytes,source_code_file)
#####
n=10
encoder_data_file="encoder_data"
ec=encoder(k,source_code,length_bytes,encoder_data_file)


In [90]:
bec=bec_channel(0.2)
bec.transfer(128,3,"encoder_dataload_A")


In [108]:
dec=decoder(128,3,"after_bec_encoder",file_name="decoder_data",source_data_file="source_code_test")


84


In [124]:
from collections import defaultdict
empty=set()
a=defaultdict(set)
a[1].add(2)
a[2].add(3)
a[1].remove(2)
a[2]==empty

False

In [135]:
a=set([1,4,3])
b=a.pop()
b=a.pop()
a

{4}

In [1]:
'''galss elimation xor
@parameter n:encoding_vector_number|receiving_vector_number
@parameter k:the source_vector number
n>=k
@parameter Y:the receive_encoding_vector
@parameter A:the  Corresponding encoding matrix

'''


1


In [10]:
import random
import numpy as np
class int2(int):
    def __add__(self,val):
        return self^val
    def __mul__(self,val):
        return self^val
class decoding_xor:
    def __init__(self,n,k,length):
        '''
        A is corresponding with Y,when change the A,we need change the Y the same time
        @parameter self.length
        @parameter self.n
        @parameter self.k
        '''
        self.n=n
        self.k=k
        self.length=length
    def random_generate(self,):
        n=self.n
        k=self.k
        length=self.length
        A=[]
        for i in range(n):
            now=[]
            for j in range(k):
                now.append(int2(random.randint(0,1)))
            A.append(now)
        self.A=np.array(A,dtype=int2)

        Y=[]
        for i in range(n):
            now=[]
            for j in range(k):
                now.append(int2(random.randint(0,1)))
            Y.append(now)
        self.Y=np.array(Y,dtype=int2)
        print(self.Y.shape)
        self.genereate_degree()
    def genereate_degree(self,):
        self.X=np.zeros((self.k,self.length),dtype="int")
        self.Y_degree=np.array([(self.Y[i]==1).sum() for i in range(self.n)])
        self.X_finished=np.zeros((self.k),dtype="int")

    def load_A_Y(self,A,Y):
        if (self.n,self.k)!=A.shape:
            raise Exception("the size is not compatable")
        if (self.n,self.length)!=Y.shape:
            raise Exception("the size of the Y is not compatable")
        self.genereate_degree()
    
    
    def gal_eli(self):
        '''
        这个高斯消元和其他的有一点点不一样。因为元素只有1
        算法思路，遍历对角线也就是k，如果此时A[k][k]!=0,在下面的k-n里面找直到找到。
        然后对下面所有的进行行操作。相当于进行了k**2的k个行操作，所以是k**3，也可以看作k*k*（n+k）的操作
        '''
        A=self.A
        Y=self.Y
        n=self.n
        k=self.k
        ##消元，变成上三角
        for row in range(k):
            if A[row][row]!=1:
                ## swwap the first 
                for row1 in range(row+1,k):
                    if A[row1][row]==1:
                        
                        A[row1],A[row]=A[row],A[row1]
                        
                        break
                    else:
                        continue
            if A[row][row]!=1:
                continue

            ### start 移位操作
        ##回代，将上三角矩阵开始往回带。
        

                
        pass
    def bp(self,):
        ###比较简单。


### 仿真结构



In [12]:
dd.random_generate()

(10, 5)


In [16]:
dd.Y

array([[1, 0, 1, 1, 0],
       [1, 0, 0, 0, 0],
       [1, 1, 0, 1, 0],
       [1, 1, 0, 0, 0],
       [0, 1, 1, 0, 0],
       [0, 1, 1, 1, 1],
       [1, 1, 1, 0, 1],
       [0, 0, 0, 1, 1],
       [0, 1, 1, 0, 0],
       [0, 0, 1, 1, 0]], dtype=object)

0
1.0
13.0
-11.0
